# FLOW_API Demonstration

## FLOW project meeting

### Leuven, 10-11 Sep 2024

In [1]:
%matplotlib inline
import os
import xarray as xr
import matplotlib.pyplot as plt
from ncplot import view

from windIO.utils.yml_utils import validate_yaml
from windIO.utils import plant_schemas_path

from flow_api import run_foxes, run_pywake #, run_wayve, run_code_saturne

## Currently available example cases

In [2]:
!(cd ../examples/cases && tree -L 1)

.
├── AWAKEN
├── KUL_LES
├── open_source_scada
├── windio_4turbines
├── windio_4turbines_ABL
├── windio_4turbines_ABL_stable
├── windio_4turbines_multipleTurbines
└── windio_4turbines_profiles_stable

9 directories, 0 files


## Example 1: Four turbines in a row, homogeneous inflow timeseries

This case is called "windio_4turbines". The input data file structure looks like this:

In [3]:
!(cd ../examples/cases/windio_4turbines && tree)

.
├── plant_energy_resource
│   ├── FLOW_toy_study_energy_resource.yaml
│   └── Stochastic_atHubHeight.nc
├── plant_energy_site
│   └── FLOW_toy_study_energy_site.yaml
├── plant_energy_turbine
│   └── DTU_10MW_turbine.yaml
├── plant_wind_farm
│   └── FLOW_toy_study_wind_farm.yaml
└── wind_energy_system
    ├── analysis.yaml
    ├── system.yaml
    ├── system_noFlowField.yaml
    ├── system_noXYgrid.yaml
    ├── system_runAll.yaml
    └── system_zplaneList.yaml

6 directories, 11 files


The main file of this case is called "FLOW_toy_study_wind_energy_system.yaml". It follows the windio schema. Notice how other files are referenced via the `!include` command:

In [4]:
!cat ../examples/cases/windio_4turbines/wind_energy_system/system.yaml

name: FLOW UQ vnv study on toy problem, 4 WT Wind Farm
site: !include ../plant_energy_site/FLOW_toy_study_energy_site.yaml
wind_farm: !include ../plant_wind_farm/FLOW_toy_study_wind_farm.yaml
attributes:
  flow_model:
    name: foxes
  analysis: !include analysis.yaml

  model_outputs_specification:
    output_folder: "results"
    #
    cases_run:
      all_occurences: False
      subset: [0, 2, 5]
    #
    turbine_outputs:
      turbine_nc_filename: 'turbine_data.nc' # dimension = states, turbine
      output_variables: ['power', 'rotor_effective_velocity'] #'frequency'
    #
    flow_field:
      report: True
      flow_nc_filename: flow_field.nc
      output_variables: ['wind_speed', 'wind_direction']
      z_planes:
        z_sampling: "hub_heights"
        xy_sampling: "grid"
        x_bounds: [-1000, 5000]
        y_bounds: [-1000, 1000]
        dx: 150
        dy: 150


This is the "analysis" input file:

In [5]:
!cat ../examples/cases/windio_4turbines/wind_energy_system/analysis.yaml

#pywake and foxes
wind_deficit_model:
  name: Bastankhah2014
  wake_expansion_coefficient: # k = ka*ti + kb
    k_a: 0.0
    k_b: 0.04
    free_stream_ti: false
  ceps: 0.2
  use_effective_ws: true
axial_induction_model: Madsen
deflection_model:
  name: None
turbulence_model:
  name: CrespoHernandez
superposition_model:
  ws_superposition: Linear
  ti_superposition: Squared
rotor_averaging:
  grid: grid
  n_x_grid_points: 4
  n_y_grid_points: 4
  background_averaging: center
  wake_averaging: center
  wind_speed_exponent_for_power: 3
  wind_speed_exponent_for_ct: 2
blockage_model:
  name: None

#wayve
layers_description:
  farm_layer_height: 150
  number_of_layers: 10
APM_additional_terms:
  term_list: None
wake_tool: "foxes"

#code_saturne
run_type: "simulate" #"postprocess"
#
HPC_config:
  run_node_number: 1
  run_ntasks_per_node: 48
  run_wall_time_hours: 6
  run_partition: ""
  #
  mesh_node_number: 1
  mesh_ntasks_per_node: 48
  mesh_wall_time_hours: 1
  mesh_partition: ""
  #
  w

We can validate the input with windIO's `validate` function:

In [6]:
input_yaml = "../examples/cases/windio_4turbines/wind_energy_system/system.yaml"
validate_yaml(input_yaml, plant_schemas_path + 'wind_energy_system.yaml')

Validation succeeded


The `flow_model` under `attributes` states `foxes`, which means that a call of the main `flow_api` would ask `foxes` to compute results. However, this choice can be overruled by specified functions/commands.

**Provided python functions:**
- *run_api(input_yaml)*: Run the case with the flow model specified in the yaml file
- *run_foxes(input_yaml)*: Run the case with `foxes`
- *run_pywake(input_yaml)*: Run the case with `PyWake`
- *run_wayve(input_yaml)*: Run the case with `WAYVE`
- *run_code_saturne(input_yaml)*: Run the case with `code_saturne`

**Provided command-line tools:**
- *flow_api input_yaml*: Run the case with the flow model specified in the yaml file
- *flow_api_foxes input_yaml)*: Run the case with `foxes`
- *flow_api_pywake input_yaml)*: Run the case with `PyWake`
- *flow_api_wayve input_yaml)*: Run the case with `WAYVE`
- *flow_api_code_saturne input_yaml)*: Run the case with `code_saturne`

For example, we can run the main API call like as follows (Note that the "!" is only needed because we are in a notebook here, not a terminal):

In [7]:
!flow_api ../examples/cases/windio_4turbines/wind_energy_system/system.yaml

Validation succeeded
Reading windio data
  Name: FLOW UQ vnv study on toy problem, 4 WT Wind Farm
  Contents: ['site', 'wind_farm', 'attributes']
Reading site
  Name: FLOW UQ vnv study on toy problem, 4 WT Wind Farm
  Contents: ['boundaries', 'energy_resource']
Ignoring 'z0', since no reference_height found. No ABL profile activated.

--------------------- Reading foxes parameter dict ---------------------
Working directory  : .
Input directory    : ../examples/cases/windio_4turbines/wind_energy_system
Output directory   : results
Initializing engine: DefaultEngine(n_procs=16, chunk_size_states=None, chunk_size_points=None)
------------------------------------------------------------------------

Running calc_farm
DefaultEngine: Selecting engine 'single'
SingleChunkEngine: Calculating 1000 states for 4 turbines
SingleChunkEngine: Running single chunk calculation for 1000 states

Running output 0: StateTurbineTable
Running function StateTurbineTable.get_dataset
Writing file results/turb

Notice that the outputs that were defined in the `yaml` above triggered the writing of two output files:

In [8]:
!tree results

results
├── flow_field.nc
└── turbine_data.nc

1 directory, 2 files


In [9]:
view("results/flow_field.nc", vars= "wind_speed")

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead


:DynamicMap   [state]
   :Image   [x,y]   (wind_speed)

In [10]:
view("results/turbine_data.nc")

:DynamicMap   [Variable]
   :QuadMesh   [time,turbine]   (power)

## Comparison of foxes and PyWake

Let's re-run `foxes`, but now a version that does not export the flow field, and now using the Python function:

In [11]:
input_yaml = "../examples/cases/windio_4turbines/wind_energy_system/system_noFlowField.yaml"
validate_yaml(input_yaml, plant_schemas_path + 'wind_energy_system.yaml')

Validation succeeded


In [12]:
!rm -rf results
run_foxes(input_yaml)

Reading windio data
  Name: FLOW UQ vnv study on toy problem, 4 WT Wind Farm
  Contents: ['site', 'wind_farm', 'attributes']
Reading site
  Name: FLOW UQ vnv study on toy problem, 4 WT Wind Farm
  Contents: ['boundaries', 'energy_resource']
Ignoring 'z0', since no reference_height found. No ABL profile activated.

--------------------- Reading foxes parameter dict ---------------------
Working directory  : .
Input directory    : ../examples/cases/windio_4turbines/wind_energy_system
Output directory   : results
Initializing engine: DefaultEngine(n_procs=16, chunk_size_states=None, chunk_size_points=None)
------------------------------------------------------------------------

Running calc_farm
DefaultEngine: Selecting engine 'single'
SingleChunkEngine: Calculating 1000 states for 4 turbines
SingleChunkEngine: Running single chunk calculation for 1000 states

Running output 0: StateTurbineTable
StateTurbineTable: Creating output dir results
Running function StateTurbineTable.get_dataset

(<xarray.Dataset> Size: 840kB
 Dimensions:     (state: 1000, turbine: 4)
 Coordinates:
   * state       (state) float64 8kB 0.0 1.0 2.0 3.0 ... 996.0 997.0 998.0 999.0
 Dimensions without coordinates: turbine
 Data variables: (12/27)
     AMB_CT      (state, turbine) float64 32kB 0.814 0.814 0.814 ... 0.814 0.814
     AMB_P       (state, turbine) float64 32kB 7.056e+06 7.056e+06 ... 7.656e+06
     AMB_REWS    (state, turbine) float64 32kB 10.09 10.09 10.09 ... 10.36 10.36
     AMB_REWS2   (state, turbine) float64 32kB 10.09 10.09 10.09 ... 10.36 10.36
     AMB_REWS3   (state, turbine) float64 32kB 10.09 10.09 10.09 ... 10.36 10.36
     AMB_RHO     (state, turbine) float64 32kB 1.225 1.225 1.225 ... 1.225 1.225
     ...          ...
     YAW         (state, turbine) float64 32kB 271.8 271.8 271.8 ... 267.1 267.1
     order       (state, turbine) int64 32kB 0 1 2 3 0 1 2 3 ... 0 1 2 3 0 1 2 3
     order_inv   (state, turbine) int64 32kB 0 1 2 3 0 1 2 3 ... 0 1 2 3 0 1 2 3
     order_ssel

In [13]:
foxes_dat = xr.load_dataset('./results/turbine_data.nc')
foxes_dat 

<xarray.Dataset> Size: 248B
Dimensions:                   (time: 3, turbine: 4)
Coordinates:
  * time                      (time) float64 24B 0.0 2.0 5.0
  * turbine                   (turbine) int64 32B 0 1 2 3
Data variables:
    power                     (time, turbine) float64 96B 7.056e+06 ... 3.24e+06
    rotor_effective_velocity  (time, turbine) float64 96B 10.09 8.287 ... 7.768

We now run `PyWake` on the same input, by using the `run_pywake` function. It writes to a folder called `output`:

In [14]:
!rm -rf output && mkdir output
run_pywake(input_yaml)

TypeError: asarray() got an unexpected keyword argument 'copy'

In [ ]:
pywake_dat = xr.load_dataset('output/PowerTable.nc')
pywake_dat

Now that all results have been computer - let's compare the outcome:

In [ ]:
plt.plot(pywake_dat.turbine, pywake_dat.power.mean('time'),c='orange', label="pywake")
plt.plot(foxes_dat.turbine, foxes_dat.power.mean('time'), ls='-.',c='blue', label="foxes")
plt.xlabel('Turbine Index')
plt.ylabel('Mean power [W]')
plt.legend()
plt.show()

In [ ]:
plt.scatter(pywake_dat.wd, pywake_dat.sel(turbine=1).power, c='orange', marker='o',alpha=0.8, label="pywake")
plt.scatter(pywake_dat.wd, foxes_dat.sel(turbine=1).power, c='blue', marker='x',alpha=0.5, label="foxes")
plt.xlabel('Wind direction [°]')
plt.ylabel('Mean power [W]')
plt.legend()
plt.show()

## Example 2: Four turbines, vertical wind inflow profile

We now want to look at the effect of averaging a background wind profile over the rotor disc area. This is the inflow data, applied to the same four-turbine wind farm of above:

In [ ]:
view("../examples/cases/windio_4turbines_profiles_stable/plant_energy_resource/Stochastic_nieuwstadt_profiles.nc")

First we wish to compute results by evaluating the background at the rotor centre points only. The wakes are evaluated on a weighted 7 x 7 grid of points. These are the corresponding windio yaml files:

In [ ]:
input_yaml_centre = "../examples/cases/windio_4turbines_profiles_stable/wind_energy_system/system.yaml"
validate_yaml(input_yaml_centre, plant_schemas_path + 'wind_energy_system.yaml')

In [ ]:
!head -n 8 ../examples/cases/windio_4turbines_profiles_stable/wind_energy_system/system.yaml

In [ ]:
!head -n 27 ../examples/cases/windio_4turbines_profiles_stable/wind_energy_system/analysis.yaml

In [ ]:
!rm -rf results
run_foxes(input_yaml_centre)

In [ ]:
centre_dat = xr.load_dataset('./results/turbine_data.nc')

Now we re-run this, using the 7 x 7 grid also for the background averagine. These are the input yaml files, notice the difference in the rotor averaging:

In [ ]:
input_yaml_grid = "../examples/cases/windio_4turbines_profiles_stable/wind_energy_system/system_grid.yaml"
validate_yaml(input_yaml_grid, plant_schemas_path + 'wind_energy_system.yaml')

In [ ]:
!head -n 8 ../examples/cases/windio_4turbines_profiles_stable/wind_energy_system/system_grid.yaml

In [ ]:
!head -n 27 ../examples/cases/windio_4turbines_profiles_stable/wind_energy_system/analysis_grid.yaml

In [ ]:
!rm -rf results/turbine_data_grid.nc
run_foxes(input_yaml_grid)

In [ ]:
grid_dat = xr.load_dataset('./results/turbine_data_grid.nc')

This shows a comparison of the time-averaged power results of the four turbines:

In [ ]:
P_centre = centre_dat.power.mean("time")
P_grid = grid_dat.power.mean("time")
plt.plot(centre_dat.turbine, P_centre, label="centre")
plt.plot(grid_dat.turbine, P_grid, label="grid49")
plt.legend()
plt.xlabel("Turbine index")
plt.ylabel("Mean power [W]")
plt.show()

## Example 3: A wind farm with 200 turbines

In [ ]:
input_yaml = "../examples/cases/open_source_scada/wind_energy_system/system.yaml"
validate_yaml(input_yaml, plant_schemas_path + 'wind_energy_system.yaml')

In [ ]:
view("../examples/cases/open_source_scada/plant_energy_resource/Stochastic_atHubHeight.nc")

In [ ]:
!rm -rf results
run_foxes(input_yaml)
foxes_dat = xr.load_dataset('./results/turbine_data.nc')

In [ ]:
view("results/turbine_data.nc")